In [121]:
%matplotlib inline
import numpy as np
import json
import pandas as pd
import geopandas
import geocoder
from geopandas.tools import sjoin
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import seaborn as sns
color = sns.color_palette()
import matplotlib.pyplot as plt

In [40]:
columns = ['road', 'city', 'neighbourhood', 'postcode', 'county', 'suburb', 'state']
#data = 
count = 0
success = 0
for i, lat_long in enumerate(train):
    lat, long = lat_long
    # already got this
    if i < len(data) and data[i][0] != "_placeholder":
        if success % 1000 == 0:
            print (success)
        success += 1
        continue
    if i < len(data):
        if success % 1000 == 0:
            print (success)
        success += 1
        continue
        
    try:
        location = geolocator.reverse("{0}, {1}".format(lat, long)).raw['address']
        new_row = [location[key] for key in info_train]
        data = np.concatenate((data, [new_row]), axis=0)
        if success % 1000 == 0:
            print (success)
            df = pd.DataFrame(data, columns=columns)
            df.to_json('data/neighborhoods')
        success += 1
    except Exception:
        if count % 100 == 0:
            print (count)
        count += 1
        new_row = ["_placeholder" for key in info_train]
        data = np.concatenate((data, [new_row]), axis=0)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
0
100
200
300
400


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
info_test = {
    'city': [],
    'county': [],
    'postcode': [],
    'road': [],
    'state': [],
    'suburb': [],
    'neighbourhood': []
}
count = 0
success = 0
for lat, long in test:
    try:
        location = geolocator.reverse("{0}, {1}".format(lat, long)).raw['address']
        for key in info_test:
            info_test[key].append(location[key] if key in location else -1)
        if success % 1000 == 0:
            print (success)
        success += 1
    except Exception:
        if count % 500 == 0:
            print (count)
        count += 1
        for key in info_test:
            info_test[key].append("_placeholder")

In [ ]:
# try using unique streets as a feature, and price per street
# maybe then we can lump streets into neighborhoods and only request once per street?

In [ ]:
# check https://www.kaggle.com/tavoloperuno/two-sigma-connect-rental-listing-inquiries/interest-level-sentiment-across-nyc-neighbourhoods/notebook
# for lumping coords into neighborhoods

# Fill in unknown Long/Lat from address

In [11]:
from geopy.exc import GeocoderTimedOut
train = pd.read_json('data/test.json')

lats = []
longs = []
for lat, long, address in train[['latitude', 'longitude', 'street_address']].values:
    if lat == 0. or long == 0.:
        while True:
            try:
                location = geolocator.geocode(address)
            except GeocoderTimedOut:
                print ('timeout, retry')
                continue
            break
        lats.append(location.latitude)
        longs.append(location.longitude)
        print ((location.latitude, location.longitude))
    else:
        lats.append(lat)
        longs.append(long)

train['latitude'] = lats
train['longitude'] = longs

train.to_json('data/test_transformed.json')

GeocoderServiceError: [Errno 65] No route to host